In [2]:
import tensorflow as tf
print("NUM GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2023-11-07 22:42:27.732652: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 22:42:27.876304: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-07 22:42:27.876381: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-07 22:42:27.877830: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-07 22:42:27.952305: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-07 22:42:27.955115: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

NUM GPUs Available:  0


2023-11-07 22:42:32.330789: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-07 22:42:32.331537: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [4]:
# data processing
# x_train is like past n days, y_train is future n days
# past n days will be 365 ? future n days is 7 ? for the week?
import pickle
import numpy as np
import pandas as pd
infile = open('data/aqi_by_county_data/cleaned_aqi_data.pkl', 'rb')
aqi_data = pickle.load(infile)
full_set = pd.DataFrame(aqi_data['AQI'])
training_set = full_set.iloc[:1676, :]
test_set = full_set.iloc[1676:, :]


In [6]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
training_set_scaled = sc.fit_transform(training_set)

x_train = []
y_train = []
n_future = 4 # next 4 days temperature forecast
n_past = 30 # Past 30 days
for i in range(0,len(training_set_scaled)-n_past-n_future+1):
    x_train.append(training_set_scaled[i : i + n_past , 0])
    y_train.append(training_set_scaled[i + n_past : i + n_past + n_future , 0 ])
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1) )

In [ ]:
# data visualization

In [ ]:
# data standardizing

In [7]:
# model: recurrent neural network

# x_train and y_train should be numpy arrays

# https://medium.com/analytics-vidhya/weather-forecasting-with-recurrent-neural-networks-1eaa057d70c3

from keras.models import Sequential
from keras.layers import Bidirectional, LSTM,Dense ,Dropout


In [8]:

# set up regressor model
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (x_train.shape[1],1) ) ))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30 , return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = n_future,activation='linear'))
regressor.compile(optimizer='adam', loss='mean_squared_error',metrics=['acc']) # adam optimizer for optimal efficiency
regressor.fit(x_train, y_train, epochs=500,batch_size=32 )


Epoch 1/500
52/52 [==============================] - 14s 95ms/step - loss: 0.0326 - acc: 0.2678
Epoch 2/500
52/52 [==============================] - 5s 91ms/step - loss: 0.0195 - acc: 0.2568
Epoch 3/500
52/52 [==============================] - 5s 90ms/step - loss: 0.0191 - acc: 0.2508
Epoch 4/500
52/52 [==============================] - 5s 90ms/step - loss: 0.0177 - acc: 0.2489
Epoch 5/500
52/52 [==============================] - 5s 90ms/step - loss: 0.0168 - acc: 0.2544
Epoch 6/500
52/52 [==============================] - 5s 89ms/step - loss: 0.0168 - acc: 0.2471
Epoch 7/500
52/52 [==============================] - 5s 89ms/step - loss: 0.0159 - acc: 0.2635
Epoch 8/500
52/52 [==============================] - 5s 91ms/step - loss: 0.0162 - acc: 0.2416
Epoch 9/500
52/52 [==============================] - 5s 92ms/step - loss: 0.0153 - acc: 0.2416
Epoch 10/500
52/52 [==============================] - 5s 91ms/step - loss: 0.0156 - acc: 0.2368
Epoch 11/500
52/52 [============================

In [9]:
# read test dataset
test_set_scaled = sc.transform(test_set)
testdataset = test_set_scaled
real_temperature = test_set_scaled
testing = sc.transform(testdataset)
testing = np.array(testing)
testing = np.reshape(testing,(testing.shape[1],testing.shape[0],1))

# predict model
predicted_temperature = regressor.predict(testing)
predicted_temperature = sc.inverse_transform(predicted_temperature)
predicted_temperature = np.reshape(predicted_temperature,(predicted_temperature.shape[1],predicted_temperature.shape[0]))

print(predicted_temperature)

1/1 [==============================] - 0s 24ms/step
[[23.847404]
 [21.79368 ]
 [19.517614]
 [21.689926]]


In [ ]:
x_test = []
y_test = []
n_future = 4 # next 4 days temperature forecast
n_past = 30 # Past 30 days
for i in range(0,len(test_set_scaled)-n_past-n_future+1):
    x_test.append(test_set_scaled[i : i + n_past , 0])
    y_test.append(test_set_scaled[i + n_past : i + n_past + n_future , 0 ])

x_test , y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (x_test.shape[0] , x_test.shape[1], 1) )

predictions = regressor.predict(x_test)

In [ ]:
mse = np.mean(np.square(predictions - y_test))
print("Mean Squared Error: ", mse)